In [1]:
import ivy

In [2]:
ivy.set_backend('torch')

ivy.experimental.lu(ivy.array([[1,2,3],[4,5,6],[7,8,9]],dtype='float32'))

(ivy.array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 ivy.array([[1.        , 0.        , 0.        ],
        [0.1429    , 1.        , 0.        ],
        [0.57139999, 0.5       , 1.        ]]),
 ivy.array([[7.        , 8.        , 9.        ],
        [0.        , 0.85710001, 1.71430004],
        [0.        , 0.        , 0.        ]]))

In [3]:
ivy.set_backend('numpy')
ivy.experimental.lu(ivy.array([[1,2,3],[4,5,6],[7,8,9]],dtype='float32'))

(ivy.array([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.]]),
 ivy.array([[1.    , 0.    , 0.    ],
        [0.1429, 1.    , 0.    ],
        [0.5714, 0.5   , 1.    ]]),
 ivy.array([[7.        , 8.        , 9.        ],
        [0.        , 0.85710001, 1.71430004],
        [0.        , 0.        , 0.        ]]))

In [7]:
# ivy.set_backend('tensorflow')
# ivy.experimental.lu(ivy.array([[1,2,3],[4,5,6],[7,8,9]],dtype='float32'))

In [20]:
import tensorflow as tf
from typing import Optional, Tuple
def lu(
    x: tf.Tensor,
    /,
    *,
    pivot: Optional[bool] = True,
) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    n = x.shape[0]
    L = tf.eye(n)
    U = tf.identity(x)
    P = tf.eye(n)

    for k in range(n - 1):
        if pivot:
            # partial pivoting
            max_idx = tf.math.argmax(tf.abs(U[k:, k]))+ k
            print(f"Max index: {max_idx}")
            # U[[k, max_idx]] = U[[max_idx, k]]
            # swap rows of U
            # perform row swapping operation
            U = tf.tensor_scatter_nd_update(U, [[k], [max_idx]], tf.gather(U, [max_idx, k]))
            # U = tf.tensor_scatter_nd_update(U, [[k], [max_idx]], tf.gather(U, [max_idx, k]), [k, k])
            print("U after swapping rows:")
            print(U)
            if k > 0:
                L = tf.tensor_scatter_nd_update(L, [[k], [max_idx]], tf.gather(L, [max_idx, k]))
            print("L after swapping rows:")
            print(L)
            if max_idx != k:
                P = tf.tensor_scatter_nd_update(P, [[k], [max_idx]], tf.gather(P, [max_idx, k]))
            print("P after swapping rows:")
            print(P)
        # elimination
        div = U[k, k]
        # perform element-wise division and assignment operation
        new_values = tf.cast(U[k+1:, k] / div, dtype=tf.float32)
        indices = tf.stack([tf.range(k+1, L.shape[0]), tf.fill([L.shape[0]-k-1], k)], axis=1)
        L = tf.tensor_scatter_nd_update(L, indices, new_values)
        print("L after elimination:")
        print(L)
        U[k+1: , k:] -= tf.matmul(L[k+1: , k], tf.expand_dims(U[k, k:], axis=0))

        print("U after elimination:")
        print(U)
    P = tf.round(P, 4)
    L = tf.round(L, 4)
    U = tf.round(U, 4)
    # ROUND P,L,U to 4 dp if more else fine
    return P, L, U


In [21]:
arr = [[1,2,3],[4,5,6],[7,8,9]]
arr_tensor = tf.convert_to_tensor(arr)
lu(arr_tensor, pivot=True)

Max index: 2
U after swapping rows:
tf.Tensor(
[[7 8 9]
 [4 5 6]
 [1 2 3]], shape=(3, 3), dtype=int32)
L after swapping rows:
tf.Tensor(
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]], shape=(3, 3), dtype=float32)
P after swapping rows:
tf.Tensor(
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]], shape=(3, 3), dtype=float32)
L after elimination:
tf.Tensor(
[[1.         0.         0.        ]
 [0.5714286  1.         0.        ]
 [0.14285715 0.         1.        ]], shape=(3, 3), dtype=float32)


InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:MatMul]

In [22]:
import tensorflow as tf
from typing import Optional, Tuple

def lu(
    x: tf.Tensor,
    /,
    *,
    pivot: bool = True,
    out: Optional[Tuple[tf.Tensor, tf.Tensor, tf.Tensor]] = None,
) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    n = x.shape[0]
    L = tf.eye(n, dtype=x.dtype)
    U = tf.identity(x)
    if pivot:
        P = tf.eye(n, dtype=x.dtype)
    for k in range(n - 1):
        if pivot:
            max_idx = tf.argmax(tf.abs(U[k:, k])) + k
            U = tf.tensor_scatter_nd_update(
                U,
                [[k, j] for j in range(k, n)],
                tf.gather(U, [max_idx, k], axis=0),
            )
            L = tf.tensor_scatter_nd_update(
                L,
                [[k, j] for j in range(k)],
                [L[max_idx, j], L[k, j]],
            )
            if max_idx != k:
                P = tf.tensor_scatter_nd_update(
                    P,
                    [[k, j] for j in range(k)],
                    [P[max_idx, j], P[k, j]],
                )

        # elimination
        div = U[k, k]
        L = tf.tensor_scatter_nd_update(
            L,
            [[i, k] for i in range(k + 1, n)],
            U[k + 1 :, k] / div,
        )
        U = tf.tensor_scatter_nd_update(
            U,
            [[i, j] for i in range(k + 1, n) for j in range(k, n)],
            U[i, j] - L[i, k] * U[k, j],
        )
        U = tf.tensor_scatter_nd_update(
            U,
            [[i, k] for i in range(k + 1, n)],
            tf.zeros([n - k - 1], dtype=x.dtype),
        )

    P = tf.round(P, decimals=4)
    L = tf.round(L, decimals=4)
    U = tf.round(U, decimals=4)

    if out is not None:
        if len(out) != 3:
            raise ValueError("out must be a tuple of length 3")
        out[0].assign(P)
        out[1].assign(L)
        out[2].assign(U)

    return P, L, U


In [4]:
arr = [[1,2,3],[4,5,6],[7,8,9]]
arr_tf = tf.convert_to_tensor(arr)

2023-03-03 06:08:49.861258: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-03 06:08:49.861293: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-03 06:08:49.861320: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-88a773): /proc/driver/nvidia/version does not exist
2023-03-03 06:08:49.869100: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
lu(arr_tf, pivot=True)

InvalidArgumentError: Outer dimensions of indices and update must match. Indices shape: [3,2], updates shape:[2,3] [Op:TensorScatterUpdate]